# Backtesting

... the kraken-infinity-grid trading algorithm

This Notebook is available at
https://github.com/btschwertfeger/kraken-infinity-grid/tree/master/tools/backtesting/
and includes the necessary knowledge to backtest strategies of the
kraken-infinity-grid. For this, the `Backtest` class (available within the
repository) is used, which instantiates the kraken-infinity-grid and mocks the
Kraken API in order to simulate real trading experience. 

Unfortunately, the Kraken API does not offer an API endpoint for retrieving all
the historical data. Instead, they provide a ZIP file that includes all the
data. This file can be downloaded via
https://support.kraken.com/hc/en-us/articles/360047543791-Downloadable-historical-market-data-time-and-sales-.
In case the URL is not available anymore, a quick web search will guide you to
the right resources.

NOTE: This Notebook is somewhat limited, since executing this in Jupyter Lab
will not skip the `time.sleep` statements, which lead to a long execution time.
Also, the notebook doesn't seem to work with larger datasets. For this reason, I
recommend running the script as plain Python script.

The following example demonstrates how to run the `backtesting.py` script (which
can be found within the repository) and the expected output in case the full
history of XBTEUR was downloaded from Kraken (see main function of the script).

```bash
python3 backtesting.py

100%|.................................| 100000/100000 [01:24<00:00, 1185.80it/s]
********************************************************************************
Strategy: GridHODL
Final price: 633.1639
Executed buy orders: 472
Executed sell orders: 467
Final balances:
BTC: {'balance': 3.202440239999995, 'hold_trade': 0.6135451499999994}
EUR: {'balance': 489.92317716484786, 'hold_trade': 453.53667409747777}
Market price: 2517.5927290401805 EUR
********************************************************************************
````

Copyright (C) 2025 Benjamin Thomas Schwertfeger <br>
GitHub: https://github.com/btschwertfeger


In [1]:
import logging

import pandas as pd
from backtesting import Backtest

# Logging can be enabled. This will log all the events that the
# kraken-infinity-grid would show. For this, adjust the logging
# level.
logging.basicConfig(
    format="%(asctime)s %(levelname)8s | %(message)s",
    datefmt="%Y/%m/%d %H:%M:%S",
    level=logging.INFO,
)

In the following cell instantiates the Backtest class, using the configuration that should be verified.

In [2]:
bt = Backtest(
    # Customize the strategy and play around with values
    strategy_config={
        # The strategy to run, one of 'GridHODL', 'GridSell',
        # 'SWING', and 'cDCA'.
        "strategy": "GridHODL",
        # Just take any userref, it is not important which one
        # to choose, except for testing multiple strategies
        # at the same time using the same DB.
        "userref": 1,
        "name": "Backtester",
        # The interval used for placing buy and sell orders.
        "interval": 0.02,
        # The order size in quote currency.
        "amount_per_grid": 100,
        # The maximum investment that the algorithm will do.
        "max_investment": 1000,
        # The number of concurrent open buy orders. Higher
        # values catch deeper dips.
        "n_open_buy_orders": 5,
        "base_currency": "BTC",
        "quote_currency": "EUR",
        # Defining the fee is optional and would be estimated
        # using the highest maker price of the asset pair (if
        # not passed).
        "fee": 0.0026,
    },
    # For backtesting, keeping the whole data in memory is ok
    db_config={"sqlite_file": ":memory:"},
    # Define the initial balances of the portfolio. Keep in mind
    # that these assets must be prefixed with 'X' for crypto
    # assets and with 'Z' for fiat currencies.
    balances={
        "balances": {
            "XXBT": {"balance": "0.0", "hold_trade": "0.0"},
            "ZEUR": {"balance": "1000.0", "hold_trade": "0.0"},
        },
    },
)

2025/01/18 08:48:54     INFO | Initiate the Kraken Infinity Grid Algorithm instance...
2025/01/18 08:48:54     INFO | Connecting to the database...
2025/01/18 08:48:55     INFO | - Initializing tables...
2025/01/18 08:48:55     INFO | - Database initialized.


In order to run the backtest, prices must be passed to the `run` function. The following cell demonstrates how to backtest a short period of price changes.

Feel free to play around with different strategies and price ranges.

In [ ]:
try:
    await bt.run(
        prices=(
            50000.0, 49000.0, 48000.0, 47000.0, 46000.0, 45000.0, 46000.0,
            47000.0, 48000.0, 49000.0, 50000.0, 49000.0, 48000.0, 47000.0,
            46000.0, 45000.0, 46000.0, 47000.0, 48000.0, 49000.0, 50000.0,
            49000.0, 48000.0, 47000.0, 46000.0, 45000.0, 46000.0, 47000.0,
            48000.0, 49000.0, 50000.0, 60000.0,
        ),
    )
except:  # pylint: disable=bare-except # noqa: S110, E722
    pass
finally:
    await bt.instance.close()
    bt.summary()

2025/01/18 08:43:35     INFO | - Subscribed to execution channel successfully!
2025/01/18 08:43:35     INFO | - Subscribed to ticker channel successfully!
2025/01/18 08:43:35     INFO | Preparing for trading by initializing and updating local orderbook...
2025/01/18 08:43:35     INFO | - Checking asset pair parameters...
2025/01/18 08:43:35     INFO | - Checking pending transactions...
2025/01/18 08:43:35     INFO | - Create sell orders based on unsold buy orders...
2025/01/18 08:43:35     INFO | - Syncing the orderbook with upstream...
2025/01/18 08:43:35     INFO |   - Retrieving open orders from upstream...
2025/01/18 08:43:35     INFO |   - Nothing changed!
2025/01/18 08:43:35     INFO | - Orderbook initialized!
2025/01/18 08:43:35     INFO | - Checking configuration changes...
2025/01/18 08:43:35     INFO |  - Amount per grid changed => cancel open buy orders soon...
2025/01/18 08:43:35     INFO |  - Interval changed => cancel open buy orders soon...
2025/01/18 08:43:36     INFO |

********************************************************************************
Strategy: GridHODL
Final price: 60000.0
Executed buy orders: 15
Executed sell orders: 15
Final balances:
BTC: {'balance': 0.0004675800000000013, 'hold_trade': 8.673617379884035e-19}
EUR: {'balance': 999.7165458055234, 'hold_trade': 496.0992654345033}
Market price: 1027.7713458055234 EUR
********************************************************************************


---

The following cells assume that you've downloaded the full history of market data from Kraken and extracted the archive.

**Keep in mind that these files can be very large!** 

For demonstrating purposes, only a subset of the whole data is used. Feel free to experiment and explore other time frames and assets!

In [3]:
with pd.read_csv("XBTEUR.csv", header=None, chunksize=100) as reader:
    for chunk in reader:
        try:
            await bt.run(prices=chunk.iloc[:, 1].values)  # price column
        except:  # pylint: disable=bare-except # noqa: S110, E722
            pass
        finally:
            await bt.instance.async_close()
            await bt.instance.stop()
            bt.summary()
        break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:16<00:00,  1.31it/s]

********************************************************************************
Strategy: GridHODL
Final price: 89.9
Executed buy orders: 23
Executed sell orders: 17
Final balances:
BTC: {'balance': 6.679519039999998, 'hold_trade': 6.32064237}
EUR: {'balance': 398.1205049748493, 'hold_trade': 294.01999896969465}
Market price: 998.6092666708491 EUR
********************************************************************************
